## В данном блокноте показан процесс работы с чистой бд

In [ ]:
!pip install pymysql
!pip install skyfield

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.2/336.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.3/232.3 kB 13.6 MB/s eta 0:00:00


In [ ]:
import pymysql
from sqlalchemy import create_engine
from skyfield.api import load
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# Параметры подключения к базе данных
db_config = {
    "host": "sql7.freemysqlhosting.net",
    "user": "sql7755157",
    "password": "EpJR55HqCE",
    "database": "sql7755157",
    "port": 3306
}

In [ ]:
Server: sql7.freemysqlhosting.net
Name: sql7755157
Username: sql7755157
Password: EpJR55HqCE
Port number: 3306

In [ ]:
# Подключение к базе данных
try:
    connection = pymysql.connect(**db_config)
    print("Успешное подключение к базе данных!")

    cursor = connection.cursor()

    # Пример запроса: получение всех таблиц в базе данных
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()
    print("Список таблиц в базе данных:")
    for table in tables:
        print(table[0])

    cursor.close()
    connection.close()

except pymysql.MySQLError as e:
    print("Ошибка подключения:", e)

Успешное подключение к базе данных!
Список таблиц в базе данных:


In [ ]:
#проверка наличия базы данных
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()
print("Список таблиц в базе данных:")
for table in tables:
  print(table[0])
connection.commit()

Список таблиц в базе данных:
astro


In [ ]:
#просмотр данных
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

cursor.execute("SELECT * FROM astro;")
rows = cursor.fetchall()
print("Обновлённые данные в таблице 'astro':")
for row in rows:
    print(row)

cursor.close()
connection.close()

Выходные данные были обрезаны до нескольких последних строк (5000).
(9976, 24, 4, 2017, 1.6874851338701895, 11.170217373664789, 0.5685275333956975, 23.812110405784964, 1.5887274321684703, 0.3933720914073906, 3.9524713877405913, 21.05364434007792, 2.359989063446777, 13.023237243469937, -4.850992208403099, 4.489397037738463, 17.805823989002317, -22.048765789557923, 9.425131086374389, 1.544150630980333, 9.07552742129805, 20.921762107979575, 22.994801720458934, -7.3741794704726615, 30.579090846284043, 19.373732288917438, -21.20526145938757, 33.04647327592944)
(9977, 25, 4, 2017, 1.654494797330955, 10.677639741825931, 0.5699417576159881, 23.837217123627255, 1.5353153062047433, 0.3998654201489357, 4.00067852786626, 21.2006385796988, 2.365235529749151, 13.015972367285514, -4.8079833354059485, 4.494066468359477, 17.803725238334653, -22.04743463579791, 9.411784750543449, 1.5477119030846165, 9.096273191521295, 20.91915052068835, 22.99646323180483, -7.364468448839091, 30.566036881847612, 19.37360

In [ ]:
#удаление базы данных
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

cursor.execute("DROP TABLE astro;")

cursor.close()
connection.close()

## Создание датафрейма эфемерид положения планет с 1990 по 2030 год

In [ ]:
eph = load('de421.bsp')
earth = eph['earth']
planets = {
    "mercury": eph["mercury"],
    "venus": eph["venus"],
    "mars": eph["mars"],
    "jupiter": eph["jupiter barycenter"],
    "saturn": eph["saturn barycenter"],
    "uranus": eph["uranus barycenter"],
    "neptune": eph["neptune barycenter"],
    "pluto": eph["pluto barycenter"]
}

ts = load.timescale()

start_date = datetime(1990, 1, 1, 0, 0, 0)
end_date = datetime(2030, 12, 31, 0, 0, 0)

data = []

current_date = start_date
row_number = 1

while current_date <= end_date:

    t = ts.utc(current_date.year, current_date.month, current_date.day)

    row_data = {
        "row_id": row_number,
        "day": current_date.day,
        "month": current_date.month,
        "year": current_date.year
    }

    for planet_name, planet in planets.items():
        astrometric = earth.at(t).observe(planet).apparent()
        ra, dec, distance = astrometric.radec()

        row_data[f'ra_{planet_name}'] = ra.hours
        row_data[f'dec_{planet_name}'] = dec.degrees
        row_data[f'distance_{planet_name}'] = distance.au

    data.append(row_data)
    current_date += timedelta(days=1)
    row_number += 1

ephemeris_df = pd.DataFrame(data)

ephemeris_df.set_index("row_id", inplace=True)

print(ephemeris_df.head())

print(ephemeris_df.info())

        day  month  year  ra_mercury  dec_mercury  distance_mercury  \
row_id                                                                
1         1      1  1990   19.855276   -20.495090          0.782228   
2         2      1  1990   19.830565   -20.241608          0.760380   
3         3      1  1990   19.792018   -20.015361          0.740231   
4         4      1  1990   19.740006   -19.818904          0.722105   
5         5      1  1990   19.675436   -19.653813          0.706305   

         ra_venus  dec_venus  distance_venus    ra_mars  ...  distance_saturn  \
row_id                                                   ...                    
1       20.555990 -16.946374        0.307653  16.541821  ...        11.005869   
2       20.542977 -16.740105        0.303373  16.591766  ...        11.007215   
3       20.527093 -16.540099        0.299290  16.641829  ...        11.008299   
4       20.508367 -16.346732        0.295412  16.692008  ...        11.009121   
5       20.48684

In [ ]:
DB_SERVER = "sql7.freemysqlhosting.net"
DB_NAME = "sql7755157"
DB_USER = "sql7755157"
DB_PASSWORD = "EpJR55HqCE"
DB_PORT = 3306

engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_SERVER}:{DB_PORT}/{DB_NAME}")

table_name = "astro"
ephemeris_df.to_sql(table_name, con=engine, if_exists='replace', index=True, index_label='date')

print(f"Данные успешно записаны в таблицу '{table_name}' базы данных!")

Данные успешно записаны в таблицу 'astro' базы данных!


In [ ]:
Server: sql7.freemysqlhosting.net
Name: sql7755157
Username: sql7755157
Password: EpJR55HqCE
Port number: 3306

In [ ]:
query = "SELECT * FROM astro;"

astro = pd.read_sql(query, engine)

astro.to_csv('Ephemeris_data.csv')